In [ ]:
import pandas as pd
import re

In [ ]:
! pip install fasttext-langdetect

## Task

For the test task you need to extract primary topics or narratives from the provided texts. Your outputs must be interpretable, i.e. by looking at them a reader must be able to get an impression of what processed texts are about.

## Solution

In my solution I am going to show an approach and thoughts about possibe approachess rather than high metrics as they require a lot of time to tune and get to the point that we need.
So I gave a detailed descriptions for each step - what we do and why and provided examples for middle steps.

## Reading & overview on data

In [ ]:
data = pd.read_csv('/content/test_assignment_data.csv')

In [ ]:
data.head()

,fullText,pubTime
0,Бабушкинский суд столицы приговорил трех актив...,2024-02-19
1,"Бывшего совладельца сетей ""Кофе хауз"" и ""Азбук...",2024-02-19
2,Ежегодный фестиваль исторических садов пройдет...,2024-02-19
3,На улицах Москвы появилось свыше 120 светоотра...,2024-02-19
4,"Три дома, в которых в свое время жили сотрудни...",2024-02-19


In [ ]:
# more detailed look to the texts

data['fullText'].sample(5).values

array(['Президент Сербии отметил, что введение мер против Москвы было бы несправедливым по отношению к русскому народуБЕЛГРАД, 20 февраля. /ТАСС/. Президент Сербии Александар Вучич намерен сохранять независимую политику по отказу от антироссийских санкций, несмотря на колоссальное давление Запада. Об этом глава государства заявил в эксклюзивном интервью генеральному директору ТАСС Андрею Кондрашову.Читайте такжеЗапрет на импорт алмазов и контроль финансовых переводов. 12-й пакет санкций ЕС против РФ"Когда на Украине начался конфликт, я сказал, что не знаю, как будут развиваться события. И мы тогда приняли решение на государственном уровне осудить конфликт, так же как и все остальные, но мы сказали, что наша позиция - не вводить санкции [против РФ]. Я сказал это тогда, потому что мы знаем по себе, каково это, когда против вас вводят санкции. У нас есть дружественный нам народ, и было бы несправедливо поступать так с русским народом", - указал Вучич."Но я заявил сербам, что не могу этого

##### **Above** we can see an interesting part in data, named **'Tags:'**
Those Tags basically describing the topic well, but they look more like key words not the topic naming form.
Lets keep in mind that we have a hint in text and continue to look in details.


Explanation about topic modelling approaches, which we will not use and why:

- LDA (Latent Dirichlet Allocation) algo described [here](https://medium.com/analytics-vidhya/text-classification-using-lda-35d5b98d4f05) which can do topic modelling on not-labelled data. It is simple to implement, works fast - but has significant disadvantages: it uses splitting text to n-grams which means we will lose context, it needs pre-defined count of categories (which we do not know exactly) and also it can mix categories between each other (like same top-words for several categories would be the same), so basically this solution will work but not percisely at all, that's why we know the approach existis but will go with another one.
Even if we'll use only **Tags:** part for LDA, we can step in mixing categories trap.

- frequency based key words detection (advantages and disadvantages almost sane same as for LDA)
  

In [ ]:
# we can see html tags etc, so I will apply light text processing

def text_preprocessing(s):
    """
    - Remove "@name"
    - Remove other special characters
    - Remove trailing whitespace and \n \t
    """
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)

    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Replace '&amp;' with '&'
    s = re.sub(r'&amp;', '&', s)
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()

    s = re.sub(r"http\S+", "", s)
    s = re.sub(r"^\n", "", s)

    for symb in ["!", ",", ":", ";", "?"]:
      s = re.sub(rf"\{symb}\.", symb, s)

    s = re.sub(r"#\S+", "", s)
    s = s.strip()

    return s

In [ ]:
data['text'] = data['fullText'].apply(text_preprocessing)

In [ ]:
# text started to look much more readable without not needed symbols
# also we can spot that overall format of the text is kind of news from different russian news sources

data['text'].sample(5).values

array(['СИМФЕРОПОЛЬ, 21 фев – РИА Новости. Депутат Госдумы от Севастополя, член комитета по международным делам Дмитрий Белик заявил, что Запад поставками дальнобойных ракет решил идти ва-банк в украинском конфликте, пытаясь оказать психологическое давление на Россию. Ранее осведомленный источник РИА Новости сообщил, что Запад потребовал от Киева предоставить перечень целей на территории России и обосновать целесообразность нанесения по ним ударов, а эксплуатацию и применение поставляемых Украине боевых ракет повышенной дальности - осуществлять под контролем натовских специалистов. "Сегодня Запад обеспокоен не за Украину, а за свои инвестиции, которые он туда вложил. Поэтому, судя по всему, западные кураторы решили пойти ва-банк, бросив на карту все, пытаясь оказать психологическое давление на Россию. Думаю, что на данном этапе вряд ли кто-то даст Украине оружие, способное поражать центр России", - сказал Белик РИА Новости. По его словам, выбор целей на территории России подразумевает 

In [ ]:
# now let's check if all examples are written in russian language to get more precise understanding

from ftlangdetect import detect

def detect_language(text):
  doc = detect(text, low_memory=True)['lang']
  return doc

data['lang'] = data['text'].apply(detect_language)

In [ ]:
data['lang'].value_counts()

ru     11956
sah       54
tt        44
os        23
mhr       14
uk         4
bg         2
Name: lang, dtype: int64

In [ ]:
# so here we can see that mostly texts are in russian with a bit of other languages like 'sah', 'tt' etc.
# non-russian texts cover approx 1% of data
# approach we will use for modeling has a possibility to handle multilingual cases so we'll keep those texts for now

data[data['lang'] == 'sah']

,fullText,pubTime,text,lang
3504,Амма сэлиэнньэтигэр «Тэтим» түөлбэ күнүн бэлиэ...,2024-02-19,Амма сэлиэнньэтигэр Тэтим» түөлбэ күнүн бэлиэт...,sah
3505,Сунтаар улууһун Тойбохой нэһилиэгэр хорсун буо...,2024-02-19,Сунтаар улууһун Тойбохой нэһилиэгэр хорсун буо...,sah
3506,Бырабыыталыстыба сыллааҕы үлэтин отчуота Сунта...,2024-02-19,Бырабыыталыстыба сыллааҕы үлэтин отчуота Сунта...,sah
3507,Москватааҕы доруобуйа харыстабылын департамены...,2024-02-19,Москватааҕы доруобуйа харыстабылын департамены...,sah
3508,"Кыһыл хаалтыс доҕордоох,\nГорн, барабаан аргыс...",2024-02-19,"Кыһыл хаалтыс доҕордоох, Горн, барабаан аргыст...",sah
3509,Горнай улууһугар өрөспүүбүлүкэтээҕи литературн...,2024-02-19,Горнай улууһугар өрөспүүбүлүкэтээҕи литературн...,sah
3510,Мэҥэ Хаҥалас Төҥүлүтүгэр баар “Нал” кафе салай...,2024-02-19,Мэҥэ Хаҥалас Төҥүлүтүгэр баар “Нал” кафе салай...,sah
3511,Олунньу 19 күнүгэр М.Е. Николаев аатынан Өрөсп...,2024-02-19,Олунньу 19 күнүгэр М.Е. Николаев аатынан Өрөсп...,sah
3512,Биэриигэ хоһоонньутар уонна ырыа толороооччула...,2024-02-19,Биэриигэ хоһоонньутар уонна ырыа толороооччула...,sah
3513,Дьокуускайга өрөспүүбүлүкэҕэ олорор норуоттар ...,2024-02-19,Дьокуускайга өрөспүүбүлүкэҕэ олорор норуоттар ...,sah


# Modeling

**Approach**

> Step 1: Label data using Gemini Pro

1.   Prompt construction
2.   Code run, save the data


> Step 2: Analyse and group received labels

> Step 3: Split data to train and test, fine-tune transformer model to detect topic on any kind of new text


### Gemini Pro labeling

In [ ]:
# # I used Gemini Pro to label initial data with following prompt:


# PROMPT = """" Как специалист по медийному пространству и журналист, классифицируй поданный текст одной категорией,
#                     которая будет включать основную тему или нарратив,
#                     категория является тематикой текста и должна состоять из от 1 до 3 слов.
#                     Обрати внимание, что тексты являются новостными статьями либо сообщениями новостного характера, категоризируй
#                     учитывая этот факт.


#         Классифицируй поданный текст по определениию выше и возврати категорию для него:
#         """


# data['predicted_by_gemini'] = gemini_predictions

In [ ]:
data_with_gemini = pd.read_csv('/content/test_data_gemini_labeled.csv')

In [ ]:
data_with_gemini.head()

,fullText,pubTime,predicted_by_gemini
0,Бабушкинский суд столицы приговорил трех актив...,2024-02-19,происшествия
1,"Бывшего совладельца сетей ""Кофе хауз"" и ""Азбук...",2024-02-19,мошенничество
2,Ежегодный фестиваль исторических садов пройдет...,2024-02-19,садово-парковое искусство
3,На улицах Москвы появилось свыше 120 светоотра...,2024-02-19,городская инфраструктура
4,"Три дома, в которых в свое время жили сотрудни...",2024-02-19,ремонт зданий


In [ ]:
# here we can see the output received from Gemini, topics are already formulated well, but
# together with it Gemini failes to 'safety error on 3822 examples of data, howewer
# even labeled dataset shoud be enough to go further

data_with_gemini['predicted_by_gemini'].value_counts().reset_index().iloc[0:40]

,index,predicted_by_gemini
0,undefined category,3822
1,военные действия,381
2,военная операция,275
3,культура,149
4,погода,139
5,выборы,127
6,геополитика,109
7,здравоохранение,98
8,дтп,94
9,мошенничество,92


In [ ]:
# let's check if data match to the labeled topic
data_with_gemini[data_with_gemini['predicted_by_gemini'] == 'военные действия']['fullText'].sample(3).values

array(['Воздушная тревога объявлена на\xa0всей территории Украины\nВоздушная тревога объявлена на\xa0всей территории Украины. Информация о\xa0этом появилась на\xa0официальном украинском ресурсе для\xa0оповещения.\nПо данным сервиса, первые сирены прозвучали в\xa012:11 по\xa0местному времени (13:11 мск) в\xa0Киеве. Через\xa0четыре минуты красная зона распространилась на\xa0все регионы страны.\nВ ночь на\xa021 февраля на\xa0Украине сообщили о\xa0взрывах в\xa0подконтрольном Киеву Херсоне. До\xa0этого звуки взрывов также раздались в\xa0городе Чугуев в\xa0Харьковской области Украины.\n19 февраля сообщалось о\xa0серии взрывов в\xa0украинских городах Полтава и Кропивницкий. На\xa0всей территории Украины была объявлена воздушная тревога.\nВооруженные силы России стали наносить удары по\xa0украинской инфраструктуре с\xa010 октября 2022\xa0года — через\xa0два дня после теракта на\xa0Крымском мосту, за\xa0которым, по\xa0заявлению российских властей, стоят спецслужбы Украины. Атаки производятся по

In [ ]:
# let's check if data match to the labeled topic
data_with_gemini[data_with_gemini['predicted_by_gemini'] == 'фигурное катание']['fullText'].sample(3).values

array(['Тарасова заявила, что Skate Canada будет переманивать российских фигуристов\nЗаслуженный тренер СССР Татьяна Тарасова призвала не отвлекаться на\xa0инициативу Федерации фигурного катания Канады (Skate Canada), которая хочет добиться упрощения процедуры смены спортивного гражданства для\xa0фигуристов. Ее слова передает Sport24.\nSkate Canada уже внесла этот вопрос в\xa0предварительную повестку конгресса Международного союза конькобежцев (ISU), который пройдет 10–14 июня в\xa0Лас-Вегасе.\n«Эта инициатива позволит нашим спортсменам выступать не только за\xa0Канаду, а за\xa0любую другую страну. Пускай они подают свое предложение в\xa0ISU. Это их право, и не имеет никого значения, будут наши фигуристы менять спортивное гражданство или нет. Да, очередные нападки, но я бы никак не стала на\xa0это реагировать.\nНужно ждать, когда нас вернут соревнования, а не отвлекаться на\xa0это. С\xa0нашим мнением никто не считается! И в\xa0этом нет ничего хорошего. Своих спортсменов не хватает, буд

In [ ]:
# it looks like labels are quite percise in the current version, so we can continue with th approach
# let's take a look how many topics we detected so far

In [ ]:
len(data_with_gemini['predicted_by_gemini'].unique())

2193

#### Matching subtopics into topic
Looks like topic are very sparse, we will try to match simular ones into 1 topic before fine-tuning

In [ ]:
def assign_ids(text_column):
    # Convert text column to a pandas Series
    series = pd.Series(text_column)

    # Create a dictionary to store text values and their corresponding IDs
    text_to_id = {}

    # Initialize ID counter
    current_id = 1

    # Iterate over each text value in the Series
    for text in series:
        # Check if the text value already exists in the dictionary
        if text not in text_to_id:
            # If not, assign a new ID to the text value
            text_to_id[text] = current_id
            current_id += 1

    # Create a new Series to store the IDs corresponding to each text value
    ids = series.map(text_to_id)

    return ids


In [ ]:
topics_df = data_with_gemini['predicted_by_gemini'].value_counts().reset_index()

# assign id to the predicted_by_gemini column
topics_df['id'] = assign_ids(topics_df['index'])

In [ ]:
topics_df = topics_df[topics_df['index'] != 'undefined category']
topics_df = topics_df.rename({'index': 'topic', 'predicted_by_gemini': 'count'}, axis = 1)

In [ ]:
# now we see the prevalance of topics initially labeled by Gemini
percs = data_with_gemini['predicted_by_gemini'].value_counts(normalize=True).reset_index()
percs = percs[percs['index'] != 'undefined category']
percs = percs.rename({'predicted_by_gemini': 'percentage'}, axis = 1)
percs = percs.drop('index', axis = 1)

pd.concat([topics_df,percs], axis=1)

,topic,count,id,percentage
1,военные действия,381,2,0.031495
2,военная операция,275,3,0.022733
3,культура,149,4,0.012317
4,погода,139,5,0.011490
5,выборы,127,6,0.010498
...,...,...,...,...
2188,планирование парковок,1,2189,0.000083
2189,падения знаменитостей,1,2190,0.000083
2190,ветеранские организации,1,2191,0.000083
2191,приюты для животных,1,2192,0.000083


In [ ]:
! pip install sentence_transformers

In [ ]:
## now we are going to make topic less sparse and combine simular ones into one using
## CLIP model as encoder, cosine simularity as simularity metric and with assumption that there
## must be not more than 250 topics

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import torch


def map_topics(topics_df):
    # Load pre-trained CLIP model for text
    device = "cuda" if torch.cuda.is_available() else "cpu"
    encoder = SentenceTransformer('clip-ViT-B-32', device = device)

    # Encode subtopic names into embeddings
    embeddings = []
    for topic in topics_df['topic']:
      embedding_for_topic =  encoder.encode(topic, device = device, convert_to_tensor = False)
      embeddings.append(embedding_for_topic)


    # Calculate pairwise cosine similarity between embeddings
    similarities = cosine_similarity(embeddings)

    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=250)
    clusters = kmeans.fit_predict(similarities)

    # Mapping topic ids to clusters
    topics_df['cluster'] = clusters


    return topics_df


In [ ]:
df = map_topics(topics_df)

In [ ]:
# so now we have 250 topics instead of 2k sparse ones, lets take a look at few of them
df[df['cluster'] == 94]

,topic,count,id,cluster
2,военная операция,275,3,94
59,военный конфликт,22,60,94
161,военная разведка,8,162,94
191,военные технологии,6,192,94
208,военная история,6,209,94
209,военная мобилизация,6,210,94
293,военные потери,4,294,94
348,военные учения,4,349,94
364,военная технология,4,365,94
447,военная медицина,3,448,94


In [ ]:
df[df['cluster'] == 2]

,topic,count,id,cluster
510,военно-морская миссия,3,511,2
1018,военно-спортивные игры,1,1019,2
1105,военно-морские учения,1,1106,2
1356,военно-морской флот,1,1357,2


In [ ]:
df[df['cluster'] == 13]

,topic,count,id,cluster
108,реновация,11,109,13
416,реставрация,3,417,13
417,развитие региона,3,418,13
829,отмена масочного режима,1,830,13
967,реновация лагерей,1,968,13
1455,регистрация в отеле,1,1456,13
1835,реорганизация аптек,1,1836,13
2054,международные финансы,1,2055,13


In [ ]:
df[df['cluster'] == 249]

,topic,count,id,cluster
618,налоговые новости,2,619,249
675,награды и признание,2,676,249
699,налогообложение,2,700,249
770,налоговый вычет,2,771,249
974,анализ данных,1,975,249
1019,налоговые льготы,1,1020,249
1133,назначение чиновников,1,1134,249
1394,ненадлежащее оказание услуг,1,1395,249
1670,налоговые нарушения,1,1671,249


In [ ]:
## ok, so clustering works good, alhought some level of mistakes is presented
## I suggest, in real-life project to do this kind of mapping semi-manually
## first step is automatic clustering and second step is manual recheck and corrections
## now lets name the cluster by top-topic (by prevalence) name

# Finding the most popular topic in each cluster
top_topics = df.groupby('cluster').apply(lambda x: x.loc[x['count'].idxmax()]).reset_index(drop=True)

# Creating a dictionary for mapping subtopics to grouped topics
cluster_new_names = []
for _, row in df.iterrows():
    cluster = row['cluster']
    topic = top_topics[top_topics['cluster'] == cluster]['topic'].iloc[0]
    cluster_new_names.append(topic)

In [ ]:
df['cluster_name'] = cluster_new_names

In [ ]:
df.sample(5)

,topic,count,id,cluster,cluster_name
68,военная подготовка,19,69,39,военная помощь
2073,арктические технологии,1,2074,184,телекоммуникации
611,финансовые проблемы,2,612,230,финансовая помощь
1463,травма музыканта,1,1464,32,праздники
299,железнодорожный транспорт,4,300,171,театральные новости


In [ ]:
# top-topic by prevalence
df.groupby('cluster_name').sum()['count'].reset_index().sort_values(by='count', ascending=False).iloc[0:20]

<ipython-input-137-f87c2addc7cc>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('cluster_name').sum()['count'].reset_index().sort_values(by='count', ascending=False).iloc[0:20]


,cluster_name,count
36,военные действия,509
31,военная операция,348
81,здравоохранение,299
42,геополитика,237
40,выборы,190
163,погода,165
112,культура,157
132,мошенничество,156
32,военная помощь,142
206,сельское хозяйство,132


### Preparation for fine-tunning and fine-tunning

We do fine-tuning on prepared data which are already with topics to be able to predict topic on new data without Gemini usage anymore but with our own proprietary model, which would be much cheaper by costs.

In [ ]:
data_with_gemini['text'] = data_with_gemini['fullText'].apply(text_preprocessing)

In [ ]:
data_with_gemini

,fullText,pubTime,predicted_by_gemini,text
0,Бабушкинский суд столицы приговорил трех актив...,2024-02-19,происшествия,Бабушкинский суд столицы приговорил трех актив...
1,"Бывшего совладельца сетей ""Кофе хауз"" и ""Азбук...",2024-02-19,мошенничество,"Бывшего совладельца сетей ""Кофе хауз"" и ""Азбук..."
2,Ежегодный фестиваль исторических садов пройдет...,2024-02-19,садово-парковое искусство,Ежегодный фестиваль исторических садов пройдет...
3,На улицах Москвы появилось свыше 120 светоотра...,2024-02-19,городская инфраструктура,На улицах Москвы появилось свыше 120 светоотра...
4,"Три дома, в которых в свое время жили сотрудни...",2024-02-19,ремонт зданий,"Три дома, в которых в свое время жили сотрудни..."
...,...,...,...,...
12092,Активный отдых должен быть комфортным. Костром...,2024-02-21,благоустройство,Активный отдых должен быть комфортным. Костром...
12093,Возможность вырастить свой урожай при поддержк...,2024-02-21,сельское хозяйство,Возможность вырастить свой урожай при поддержк...
12094,Городской цирк костромичи выбрали во время онл...,2024-02-21,городская среда,Городской цирк костромичи выбрали во время онл...
12095,Для комфортного и безопасного образования. В ш...,2024-02-21,undefined category,Для комфортного и безопасного образования. В ш...


In [ ]:
# adding 250 groupped topics to data
c = ['topic', 'cluster_name']

data_with_gemini = data_with_gemini.merge(df[c], left_on = 'predicted_by_gemini', right_on = 'topic', how = 'left')

In [ ]:
# now we have everything we need - cleaned text and assingned topics names (cluster_name)

data_with_gemini.head()

,fullText,pubTime,predicted_by_gemini,text,topic,cluster_name
0,Бабушкинский суд столицы приговорил трех актив...,2024-02-19,происшествия,Бабушкинский суд столицы приговорил трех актив...,происшествия,происшествия
1,"Бывшего совладельца сетей ""Кофе хауз"" и ""Азбук...",2024-02-19,мошенничество,"Бывшего совладельца сетей ""Кофе хауз"" и ""Азбук...",мошенничество,мошенничество
2,Ежегодный фестиваль исторических садов пройдет...,2024-02-19,садово-парковое искусство,Ежегодный фестиваль исторических садов пройдет...,садово-парковое искусство,поисково-спасательная операция
3,На улицах Москвы появилось свыше 120 светоотра...,2024-02-19,городская инфраструктура,На улицах Москвы появилось свыше 120 светоотра...,городская инфраструктура,городская инфраструктура
4,"Три дома, в которых в свое время жили сотрудни...",2024-02-19,ремонт зданий,"Три дома, в которых в свое время жили сотрудни...",ремонт зданий,некролог


In [ ]:
data_with_gemini.to_csv('test_data_to_transformer.csv')

In [ ]:
data_with_gemini = pd.read_csv('test_data_to_transformer.csv')

#### Preprocess labels to be used as input to XLM_ROBERTA_MODEL

In [ ]:
# filter from train data those examples where Gemini didn't returned a topic

data_for_training = data_with_gemini[~data_with_gemini.topic.isna()]

In [ ]:
data_for_training.head()

,Unnamed: 0,fullText,pubTime,predicted_by_gemini,text,topic,cluster_name
0,0,Бабушкинский суд столицы приговорил трех актив...,2024-02-19,происшествия,Бабушкинский суд столицы приговорил трех актив...,происшествия,происшествия
1,1,"Бывшего совладельца сетей ""Кофе хауз"" и ""Азбук...",2024-02-19,мошенничество,"Бывшего совладельца сетей ""Кофе хауз"" и ""Азбук...",мошенничество,мошенничество
2,2,Ежегодный фестиваль исторических садов пройдет...,2024-02-19,садово-парковое искусство,Ежегодный фестиваль исторических садов пройдет...,садово-парковое искусство,поисково-спасательная операция
3,3,На улицах Москвы появилось свыше 120 светоотра...,2024-02-19,городская инфраструктура,На улицах Москвы появилось свыше 120 светоотра...,городская инфраструктура,городская инфраструктура
4,4,"Три дома, в которых в свое время жили сотрудни...",2024-02-19,ремонт зданий,"Три дома, в которых в свое время жили сотрудни...",ремонт зданий,некролог


**text** column would be our input data (X)
**cluster_name** (Y) is the grouped topic (to which we've added several subtopic on the previous stage (applicatiob of CLIP model +kmeans))

as transformer models accept Y variable ids which start with 0, we will map now **cluster_name** again to digits from 0 to 249.


In [ ]:
labels_dt = pd.DataFrame({'cluster_name' : data_for_training['cluster_name'].unique(), 'roberta_label': [i for i in range(250)]})
labels_dt

,cluster_name,roberta_label
0,происшествия,0
1,мошенничество,1
2,поисково-спасательная операция,2
3,городская инфраструктура,3
4,некролог,4
...,...,...
245,моряки,245
246,мелкое хулиганство,246
247,миграция животных,247
248,парусный спорт,248


In [ ]:
data_for_training = data_for_training.merge(labels_dt, on = 'cluster_name', how = 'left')

In [ ]:
# create sample by topic, because on full data Colab fails to OOM, just to show an example of working fine-tuning
data_for_training = data_for_training[data_for_training['roberta_label'].isin([0,1,2,3,4,5])]

In [ ]:
len(data_for_training)

376

### Fine Tuning XLM RoBERTa

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt

def evaluate_model(classes_predicted, y_true):
    # Get accuracy over the test set
    accuracy = accuracy_score(y_true, classes_predicted)
    print(f'Accuracy: {accuracy*100:.2f}%')

    # Get Confusion Matrix
    cm = confusion_matrix(y_true, classes_predicted)
    print(cm)

    return accuracy

In [ ]:
import numpy as np
import random
import json
import torch
import transformers
import torch.nn as nn

from transformers import (
    AdamW,
    AutoModel,
    AutoTokenizer,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    BertTokenizer,
    BertForSequenceClassification,
    XLMRobertaTokenizer,
    XLMRobertaForSequenceClassification
)
from transformers import TrainingArguments, Trainer
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report
)
from sklearn.model_selection import train_test_split


In [ ]:
from transformers import AutoTokenizer

MODEL = 'xlm-roberta-base'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )

        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
%%time

import torch
import torch.nn as nn
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup

class XLMRobertaFineTuner(nn.Module):
    def __init__(self, num_classes, pretrained_model_name='xlm-roberta-base'):
        super(XLMRobertaFineTuner, self).__init__()
        self.tokenizer = XLMRobertaTokenizer.from_pretrained(pretrained_model_name)
        self.model = XLMRobertaForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=num_classes)
        self.num_classes = num_classes

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits

    def encode_text(self, text_list, max_length=512, padding='max_length', truncation=True, return_tensors='pt'):
        return self.tokenizer(text_list, padding=padding, truncation=truncation, max_length=max_length, return_tensors=return_tensors)

    def get_optimizer_and_scheduler(self, learning_rate, total_steps, warmup_steps):
        optimizer = AdamW(self.model.parameters(), lr=learning_rate)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
        return optimizer, scheduler

def train(model, dataloader, optimizer, scheduler, device, num_epochs):
    model = nn.DataParallel(model, device_ids=[0, 1])
    model.train()
    model.to(device)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping to prevent exploding gradients
            optimizer.step()
            scheduler.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

CPU times: user 44 µs, sys: 0 ns, total: 44 µs
Wall time: 45.5 µs


#### Split data to train/test and tokenize

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
batch_size = 4

# Specify `MAX_LEN`
MAX_LEN = 512
num_classes = len(data_for_training['cluster_name'].unique())
learning_rate = 4e-5
total_steps = 1000
warmup_steps = 100
num_epochs = 2

dev = 'cpu'
device = torch.device(dev)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_for_training['text'], data_for_training['roberta_label'], test_size=0.5, random_state=42)


In [ ]:
# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Tokenizing data...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


#### Fine tune (train) step

In [ ]:
# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_test.values)

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)


torch.cuda.empty_cache()

model = XLMRobertaFineTuner(num_classes)
optimizer, scheduler = model.get_optimizer_and_scheduler(learning_rate, total_steps, warmup_steps)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_dataloader, optimizer, scheduler, device, num_epochs)



Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/2, Loss: 1.6783
Epoch 2/2, Loss: 1.4680


In [ ]:
import os

def save_model(model, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model_to_save = model.model.module if hasattr(model.model, 'module') else model.model
    model_to_save.save_pretrained(output_dir)
    print(f"Model saved at {output_dir}")

save_model(model, output_dir = '/content/')

Model saved at /content/


In [ ]:
def predict_via_model(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    output = model(**inputs)
    predicted_class = torch.argmax(output, axis=-1)[0]
    return int(predicted_class)

In [ ]:
classes_prediction_example = X_test[0:5].apply(predict_via_model)

In [ ]:
classes_prediction_example

6345    1
7755    5
5581    5
3277    1
2946    1
Name: text, dtype: int64

In [ ]:
y_test[0:5]

6345    1
7755    1
5581    3
3277    1
2946    0
Name: roberta_label, dtype: int64

# Conclutions

I received a set of texts witout topics and created initial topics using Prompt Engineering and Gemini model (free version) from Google, which gave stable and very good by quality results.
After that, I moved to improvement to decrease future costs, decided to use Gemini topics as labels for transformers fine-tuning, to be able to use our own model on new data.

I used very small sample and epochs = 2 for fine-tune because of Colab restrictions and long processing by time on cpu, also I've shown predict only on several examples (where 3 from 5 are correct ones).
Together with it, I've added evaluate_model() function which can calculate accuracy and prints confusion matrix on all test data in the future.

For the brief look and beginning, looks like task is resolvable by suggested approach.